In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#from EVCSIndividual import EVCSIndividual
import copy
import random
import time 

class EVCSIndividual:

    '''
    individual of demand oriented method
    '''
    def __init__(self,data,ev_ct,ec_cp,ev_n):

        self.data=data  #候选数据
        self.evcs=np.zeros(ev_n)  #每个充电站的车数
        self.individual=[]
        self.fitness=[]
        self.ev_ct=ev_ct
        self.ev_cp=ec_cp
        self.ev_n=ev_n

    def GenerateIndividual(self):
        '''
        generate a random chromsome for genetic algorithm
        '''
        #print("yes")
        gene = list()    #问题的解，基因，种群中的个体：[0，...，city_size]
        for i in range(len(self.data)):
            ind=random.choices(self.data[i])
            copy_ind=copy.deepcopy(ind[0])
            copy_ind.append(i)
            gene.append(copy_ind)
        random.shuffle(gene)
        self.individual=gene

    def calculateFitness(self,cd_n=5):
        '''
        calculate the fitness of the indiviual
        '''
        #distance = 0.0
        ev_n=np.zeros((self.ev_n,cd_n))  #单位时间内每个电桩充电车数
        cd_wt=np.zeros((self.ev_n,cd_n)) #充电桩工作总时间
        cd_st=np.zeros((self.ev_n,cd_n)) #充电桩空闲时间
        st=0.   #电桩空闲总时间  
        qt=0.   #电车排队总时间 
        tt=0.   #充电总时间 到充电桩时间+排队时间+充电时间     
        cost=0. #充电总成本成本
        n=[]  
        x_n=0
        for i in range(len(self.individual)): #len(self.indiviual)
            #cost+=self.individual[i][1]*self.ev_ct[i]
            cost+=self.individual[i][1]+self.ev_cp[self.individual[i][0]]*self.ev_ct[self.individual[i][2]]
            k=int(self.evcs[self.individual[i][0]]%cd_n)
            #print(k)
            self.evcs[self.individual[i][0]]+=1
            if cd_wt[self.individual[i][0]][k]<60:
                ev_n[self.individual[i][0]][k]+=1
            if self.individual[i][1]<cd_wt[self.individual[i][0]][k]:
                tt+=cd_wt[self.individual[i][0]][k]+self.ev_ct[self.individual[i][0]]
                qt+=cd_wt[self.individual[i][0]][k]-self.individual[i][1]
                #print("第%d辆排队的EV，车辆编号：%d,具体信息：%s,桩号：%d,需要排队时间：%f" %(x_n,i,str(self.indiviual[i]),k,cd_wt[self.indiviual[i][0]][k]-self.indiviual[i][1]))
                cd_wt[self.individual[i][0]][k]=cd_wt[self.individual[i][0]][k]+self.ev_ct[self.individual[i][0]]
                x_n+=1
            else:
                tt+=self.individual[i][1]+self.ev_ct[self.individual[i][0]]
                st+=self.individual[i][1]-cd_wt[self.individual[i][0]][k]
                cd_st[self.individual[i][0]][k]+=self.individual[i][1]-cd_wt[self.individual[i][0]][k]
                cd_wt[self.individual[i][0]][k]=self.individual[i][1]+self.ev_ct[self.individual[i][0]]
        revnue=0.
        t_ev_n=0
        t_st_v=0
        for i in range(self.ev_n):
            for j in range(cd_n):
                revnue+=(cd_wt[i][j]-cd_st[i][j])*self.ev_cp[i]
                t_ev_n+=ev_n[i][j]
                t_st_v+=cd_st[i][j]
        
        n.append(revnue)      #总收益  
        n.append(cost)        #总成本
        n.append(tt)          #总时间
        n.append(qt)          #总排队时间
        n.append(st)          #总空闲时间
        n.append(t_ev_n)      #单位时间每个充电桩充电电车总数
        self.fitness=np.array(n)

class GeneticAlgorithm:
    #print("ga")
    def __init__(self, t_c_l, ev_ct, ev_cp, n,c_rate=0.7, m_rate=0.3, pop_size=200, maxnum=3000):
        self.pop_size = pop_size#50
        self.fitness = np.zeros(self.pop_size)#Pop_size 为50，种群个数，应该修改为参数，可传递任意种群个数
        self.c_rate = c_rate #交叉率，0.7，修改为可变参数
        self.m_rate = m_rate #变异率，0.05，修改为
        self.maxiternum = maxnum# 最大迭代次数
        self.population=[]
        self.bestfitness=0.   #最好的适应值
        self.besttruefitness=[] # 最好的真适应值
        self.bestIndex=0
        self.bestgene=[]
        #self.dw = Draw()    #绘图类
        self.trace = np.zeros((self.maxiternum, 2))
        self.avefitness=0.
        self.data=t_c_l
        self.ev_ct=ev_ct
        self.ev_cp=ev_cp
        self.cs_n=n
        self.bestindividual=[]   #最好的个体
        self.dataset=[]
        self.matrix=np.zeros((len(t_c_l),self.cs_n))
    
    def initialize(self):
        '''
        initialize the population of GA
        '''
        for i in range(0, int(self.pop_size)):
            ind = EVCSIndividual(self.data,self.ev_ct,self.ev_cp,self.cs_n)
            ind.GenerateIndividual()
            self.population.append(ind)
        #print("gainitialize")
    
        
    def evaluation(self):
        '''
        evaluation the fitness of the population
        '''
        for i in range(0, int(self.pop_size)):
            self.population[i].calculateFitness()
            self.fitness[i] = 0.3*(self.population[i].fitness[3]/40000)+0.4*(self.population[i].fitness[4]/3000)+0.3*(1-self.population[i].fitness[5]/899)
            
    def selection(self):
        self.dataset.append(self.population[self.bestIndex].individual)
        for i in range(self.pop_size):
            if i != self.bestIndex and self.fitness[i] > self.avefitness:
                pi = self.cross(self.population[self.bestIndex].individual, self.population[i].individual)
                self.population[i].individual= self.mutate(pi)
                self.population[i].individual= sorted(self.population[i].individual, key=(lambda x: [x[0],x[1]]))
                self.population[i].calculateFitness()
                self.fitness[i] = 0.3*(self.population[i].fitness[3]/40000)+0.4*(self.population[i].fitness[4]/3000)+0.3*(1-self.population[i].fitness[5]/899)
        
    def selectionEDA(self):
        self.dataset.append(self.population[self.bestIndex].individual)
        for i in range(self.pop_size):
            if i != self.bestIndex:
                ind=self.makenewInd()
                nind = self.cross(self.population[self.bestIndex].individual, ind)    #交叉种群中第j,r个体的基因
                self.population[i].individual = nind   #突变种群中第j个体的基因
                self.population[i].individual= sorted(self.population[i].individual, key=(lambda x: [x[0],x[1]]))
                self.population[i].calculateFitness()
                self.fitness[i] = 0.3*(self.population[i].fitness[3]/40000)+0.4*(self.population[i].fitness[4]/3000)+0.3*(1-self.population[i].fitness[5]/899)
    
    def crossoverMutation(self):
         for j in range(self.pop_size):
                #if j != self.bestIndex:
                r = np.random.randint(0, self.pop_size - 1)
                if j != r:
                    nind = self.cross(self.population[j].individual, self.population[r].individual)    #交叉种群中第j,r个体的基因
                    self.population[j].individual = self.mutate(nind)    #突变种群中第j个体的基因
                    self.population[j].individual= sorted(self.population[j].individual, key=(lambda x: [x[0],x[1]]))
                    self.population[j].calculateFitness()
                    #self.fitness[j] = self.population[j].fitness
                    self.fitness[j] = 0.3*(self.population[j].fitness[3]/40000)+0.4*(self.population[j].fitness[4]/3000)+0.3*(1-self.population[j].fitness[5]/899)
       
    def crossoverMutationEDA(self):
         for j in range(self.pop_size):
                if j != self.bestIndex:
                    ind=self.makenewInd()
                    nind = self.cross(self.population[j].individual, ind)    #交叉种群中第j,r个体的基因#                 nind = self.cross(self.population[self.bestIndex].individual, ind)    #交叉种群中第j,r个体的基因
                    self.population[j].individual = nind   #突变种群中第j个体的基因
                    self.population[j].individual= sorted(self.population[j].individual, key=(lambda x: [x[0],x[1]]))
                    self.population[j].calculateFitness()
                    self.fitness[j] = 0.3*(self.population[j].fitness[3]/40000)+0.4*(self.population[j].fitness[4]/3000)+0.3*(1-self.population[j].fitness[5]/899)
       
    def cross(self, parent1, parent2):
        """交叉p1,p2的部分基因片段"""
        if np.random.rand() > self.c_rate:
            return parent1
        index1 = np.random.randint(0, len(parent1) - 1)
        index2 = np.random.randint(index1,  len(parent1)- 1)
        tempGene = parent2[index1:index2]  # 交叉的基因片段
        tempGene_t = pd.DataFrame(tempGene,columns=["0","1","2"])
        tempGene_c = list(tempGene_t["2"])
        newGene = []
        p1len = 0
        for g in parent1:
            if p1len == index1:
                newGene.extend(tempGene)  # 插入基因片段
            if g[2] not in tempGene_c:
                newGene.append(g)
            p1len += 1
        
        if len(newGene)!=len(parent1):
            ind = EVCSIndividual(self.data,self.ev_ct,self.ev_cp,self.cs_n)
            ind.GenerateIndividual()
            return ind.individual
        
        return newGene
    
    def crossEDA(self, parent1, parent2):
        """交叉p1,p2的部分基因片段"""
        if np.random.rand() > self.c_rate:
            return parent1
        ind=self.makenewInd()
        tempGene= sorted(ind, key=(lambda x: [x[0],x[1]]))
        #print(len(tempGene))
        newGene = []
        p1len = 0
        index1 = np.random.randint(0, len(tempGene) - 1)
        index2 = np.random.randint(index1,  len(tempGene)- 1)
        tempGene_t = pd.DataFrame(tempGene[index1:index2],columns=["0","1","2"])
        tempGene_c = list(tempGene_t["2"])

        q=random.random()
        if q>0.5:             #原0.5，测试开始
            for g in parent1:
                if p1len == index1:
                    newGene.extend(tempGene[index1:index2])  # 插入local基因片段
                if g[2] not in tempGene_c:
                    newGene.append(g)
                p1len += 1
        else:
            for g in parent2:
                if p1len == index1:
                    newGene.extend(tempGene[index1:index2])  # 插入local基因片段
                if g[2] not in tempGene_c:
                    newGene.append(g)
                p1len += 1

        if len(newGene)!=len(parent1):
            ind = EVCSIndividual(self.data,self.ev_ct,self.ev_cp,self.cs_n)
            ind.GenerateIndividual()
            return ind.individual
        
        return newGene

    
    def mutate(self,gene):
        """突变"""
        if np.random.rand() > self.m_rate:
            return gene
        index1 = np.random.randint(0, len(gene) - 1)
        index2 = np.random.randint(index1, len(gene) - 1)
        newGene = self.reverse_gen(gene, index1, index2)
        if len(newGene)!=len(gene):
            ind = EVCSIndividual(self.data,self.ev_ct,self.ev_cp,self.cs_n)
            ind.GenerateIndividual()
            return ind.individual
        return newGene
    
    def reverse_gen(self, gen, i, j):
        #函数：翻转基因中i到j之间的基因片段
        if i >= j:
            return gen
        if j > len(gen) - 1:
            return gen
        parent1 = copy.deepcopy(gen)
        tempGene = parent1[i:j]
        parent1[i:j]=tempGene[::-1]
        return parent1
    
    def computematrix(self,cd_t=5):
       # ev_t=np.zeros(self.cs_n)  #电桩充电车数
        for i in range(len(self.dataset)):
            for j in range(len(self.dataset[i])):
                cs_n=int(self.dataset[i][j][0])           #充电站序号
                ev_n=int(self.dataset[i][j][2])           #电车序号
                self.matrix[ev_n][cs_n]+=1

    def makenewInd(self):
        newgene=[]
        for i in range(len(self.matrix)):
            k=np.argmax(self.matrix[i])
            for j in range(len(self.data[i])):
                if self.data[i][j][0]==k:
                    cs=copy.deepcopy(self.data[i][j])
                    cs.append(i)
                    break
            newgene.append(cs)
        ind = EVCSIndividual(self.data,self.ev_ct,self.ev_cp,self.cs_n)
        ind.GenerateIndividual()
        newgene=self.cross(newgene,ind.individual) 
        return newgene    
    
    def solve(self):
        self.t = 0
        self.initialize()
        self.evaluation()
        self.bestfitness = np.min(self.fitness)
        self.bestIndex = np.argmin(self.fitness)
        self.bestgene = copy.deepcopy(self.population[self.bestIndex])
        self.besttruefitness=self.population[self.bestIndex].fitness
        self.bestindividual=self.population[self.bestIndex].individual
        self.avefitness = np.mean(self.fitness)
        self.trace[self.t, 0] = self.fitness[self.bestIndex]
        self.trace[self.t, 1] = self.avefitness
        print("%d次迭代: 适应值:%f,平均适应值:%f,最优结果:%s" % (
            self.t, self.trace[self.t, 0], self.trace[self.t, 1],str(self.bestgene.fitness)))

        while self.t < self.maxiternum - 2500:
            self.t += 1
            self.selection()
            self.crossoverMutation()
            localbest = np.min(self.fitness)
            self.bestIndex = np.argmin(self.fitness)
            if localbest<self.bestfitness:
                self.bestfitness=localbest
                self.besttruefitness=self.population[self.bestIndex].fitness
                self.bestindividual=self.population[self.bestIndex].individual
            self.bestgene = copy.deepcopy(self.population[self.bestIndex])
            self.avefitness = np.mean(self.fitness)            
            self.trace[self.t, 0] = self.fitness[self.bestIndex]
            self.trace[self.t, 1] = self.avefitness
            print("%d次迭代: 适应值:%f,平均适应值:%f,最优结果:%s" % (
                self.t, self.trace[self.t, 0], self.trace[self.t, 1],str(self.bestgene.fitness)))
            if self.t%25==0:
                self.population.clear()
                self.initialize()
                self.evaluation()
        
        
        print("-----------------EDA开始-------------------")    
        self.computematrix() 
        self.dataset.clear()
        while self.t < self.maxiternum -1:
            self.t += 1
            self.selectionEDA()
            self.computematrix()
            self.dataset.clear()
            self.crossoverMutation()
            localbest = np.min(self.fitness)
            if localbest<self.bestfitness:
                self.bestfitness=localbest
                self.besttruefitness=self.population[self.bestIndex].fitness
                self.bestindividual=self.population[self.bestIndex].individual
            self.bestIndex = np.argmin(self.fitness)
            self.bestgene = copy.deepcopy(self.population[self.bestIndex])
            self.avefitness = np.mean(self.fitness)
            self.trace[self.t, 0] = self.fitness[self.bestIndex]
            self.trace[self.t, 1] = self.avefitness
            print("%d次迭代: 适应值:%f,平均适应值:%f,最优结果:%s" % (
                self.t, self.trace[self.t, 0], self.trace[self.t, 1],str(self.bestgene.fitness)))
        print("True Best Fitness:",self.besttruefitness)
        print("Best Fitness:",self.bestfitness)
        print("Average of queuing time (minute):",self.besttruefitness[3]/899)
        print("Average of idle time (minute):",self.besttruefitness[4]/170)
        print("Number of charged EV within an hour:",int(self.besttruefitness[5]))
        print("Total cost of all EV:",self.besttruefitness[1])
        print("Total revenue of EVCS:",self.besttruefitness[0])

    
        
def main():
    '''读取充电站数据'''
    data = pd.read_csv("chargingstations.csv", delimiter=";", header=None).values  #读取34省市数据，以';'分割，dataframe

    cities = data[:, 1:]# 0行至最后一行，1列至最后一列
    cities_name = data[:, 0] #0行至最后一行，第0列
    city_size = data.shape[0]# shape属性返回包含行和列的元组
    locations = np.arange(cities.shape[0])    #问题的解，基因，种群中的个体：[0，...，city_size]

    ev_data_1000= pd.read_csv('ev_data_1000.csv').values 
    ev_x=ev_data_1000[:, 1:2]
    ev_x=[j for i in ev_x for j in i]
    ev_y=ev_data_1000[:, 2:3]
    ev_y=[j for i in ev_y for j in i]
    ev_ld=ev_data_1000[:, 3:4]
    ev_ld=[j for i in ev_ld for j in i]
    ev_ct=ev_data_1000[:, 4:5]
    ev_ct=[j for i in ev_ct for j in i]
    ev_cp_34= pd.read_csv('ev_cp_34.csv').values 
    ev_cp=ev_cp_34[:,1:]
    ev_cp=[j for i in ev_cp for j in i]

    t_c_l=[] #每个电车的候选充电站
    for i in range(1000):
        c_l=[]
        for j in range(34):
            d=np.sqrt((ev_x[i] - cities[j][0]) ** 2 + (ev_y[i] - cities[j][1]) ** 2)
            #d=ct_distance(ev_x[i],ev_y[i],cities[j])
            if d<ev_ld[i]:
                c=[]
                c.append(int(j))      # 充电站序号
                c.append(d)      # 距离充电站距离，也是行驶时间
                #c.append(ev_cp[j])  #充电电价
                c_l.append(c)
        t_c_l.append(c_l)        

    for i in range(len(t_c_l)-1,-1,-1):
        if t_c_l[i]==[]:
            del ev_x[i]
            del ev_y[i]
            del ev_ld[i]
            del ev_ct[i]
            del t_c_l[i]

    GA=GeneticAlgorithm(t_c_l,ev_ct,ev_cp,len(cities))
    GA.initialize()
    GA.solve()
        
#目前最优[ 16575.94745795 175006.78616601  55609.61504463  30785.9283487   2206.67604534    634.  ]
#适应值:0.594212,平均适应值:0.659067,最优结果:[ 16770.97487788 171025.98243471  55252.89521337  30548.93880873  2089.99381702    640. ]
#适应值:0.508712,平均适应值:0.521612,最优结果:[ 16763.10019171 170352.63143501  53207.77215977  28552.77795326  1608.58002773    659. ]
#适应值:0.507338,平均适应值:0.526123,最优结果:[ 16759.53057865 174122.93111212  52971.02862662  28044.18074583  1671.93032886    677. ]
#适应值:0.509895,平均适应值:0.526672,最优结果:[ 16975.6548079  172928.44553477  53425.33019926  28478.60787967  1614.11138817    656. ]
#适应值:0.506314,平均适应值:0.520382,最优结果:[ 16700.56529068 175292.11579819  52928.8048893   27954.31953142  1641.78015438    666. ]
#适应值:0.341276,平均适应值:0.426530,最优结果:[ 16732.14462081 172822.0074459   48935.49296481  24094.80858149  721.19960212     706. ]
#适应值:0.308162,平均适应值:0.374562,最优结果:[ 16572.98572384 172427.23843528  46787.78056479  22099.75340968   645.13464115    730. ]  
#适应值:0.298703,平均适应值:0.363658,最优结果:[ 16507.57790893 173259.70236797  46192.59549067  21596.93266218   655.03394976    751. ]
#适应值:0.250530,平均适应值:0.307847,最优结果:[16256.40138289 28624.70955899 43462.75668406 18727.39244804    490.18627142   765.     ]
#适应值:0.242920,平均适应值:0.298644,最优结果:[16238.32761929 28845.78642801 43172.33881781 18214.57850617    479.47823332   772.     ]
#适应值:0.226712,平均适应值:0.291798,最优结果:[16233.56430733 29099.41994284 42717.90021253 17520.3704055     414.48271128    779.     ]
#适应值:0.268526,平均适应值:0.276436,最优结果:[16347.44974434 28592.91269038 44928.82728407 20168.7029072    539.07936596   763.        ]
#适应值:0.258384,平均适应值:0.264783,最优结果:[16233.26541666 28581.34319974 43890.13072447 19205.90358073   527.17783914   767.        ]
#适应值:0.213003,平均适应值:0.220379,最优结果:[16176.42992451 29045.84509205 42230.33857752 17134.62138839   340.87243395   782.        ]
if __name__ == '__main__':
    start=time.time()
    main()
    end=time.time()
    print("Time consuming:",end-start)

0次迭代: 适应值:0.611938,平均适应值:0.658879,最优结果:[16595.36657671 28309.5370598  56582.63352714 31916.0211406
  2138.53306889   637.        ]
1次迭代: 适应值:0.334182,平均适应值:0.365049,最优结果:[16615.09073928 28160.39244141 48088.94188204 23457.17468136
   723.88620779   714.        ]
